In [1]:
import acl
import numpy as np
import struct
from constant import ACL_MEM_MALLOC_NORMAL_ONLY, \
    ACL_MEMCPY_HOST_TO_DEVICE, ACL_MEMCPY_DEVICE_TO_HOST, \
    ACL_ERROR_NONE, NPY_BYTE

In [2]:
from acl_sample import Sample

In [3]:
sample = Sample(0,
                "./openpose_from_tf.om",
                368,
                368)

[Sample] init resource stage:
[Sample] init resource stage success
[Model] class Model init resource stage:
model output size 3
output  0
model output dims ({'name': 'peaks:0', 'dimCount': 4, 'dims': [1, 92, 92, 19]}, 0)
model output datatype 0
output  1
model output dims ({'name': 'heatmat_output:0', 'dimCount': 4, 'dims': [1, 92, 92, 19]}, 0)
model output datatype 0
output  2
model output dims ({'name': 'pafmat_output:0', 'dimCount': 4, 'dims': [1, 92, 92, 38]}, 0)
model output datatype 0
[Model] create model output dataset:
[Model] create model output dataset success
[Model] class Model init resource stage success
input tensor size 4000
output tensor size 2000


In [4]:
img_dict = {"path": "./golf.jpg", "dtype": np.uint8}

In [5]:
model_output = sample.forward(img_dict)

[Sample] width:1313 height:1348
[Sample] image:./golf.jpg


Exception: acl.rt.memcpy failed ret=200001

In [41]:
num_output = acl.mdl.get_dataset_num_buffers(model_output)

def get_model_output_by_index(model_output, i):
    temp_output_buf = acl.mdl.get_dataset_buffer(model_output, i)

    infer_output_ptr = acl.get_data_buffer_addr(temp_output_buf)
    infer_output_size = acl.get_data_buffer_size(temp_output_buf)
#     print("infer_output_size", infer_output_size)
    
    output_host, _ = acl.rt.malloc_host(infer_output_size)
    acl.rt.memcpy(output_host, infer_output_size, infer_output_ptr,
                          infer_output_size, ACL_MEMCPY_DEVICE_TO_HOST)
    
    result = acl.util.ptr_to_numpy(output_host, (infer_output_size,), NPY_BYTE)
    return np.array(struct.unpack(f"{infer_output_size//4}f", bytearray(result))).reshape(92, 92, -1)

In [52]:
peaks = get_model_output_by_index(model_output, 0)
heatmat = get_model_output_by_index(model_output, 1)
pafmat = get_model_output_by_index(model_output, 2)